In [1]:
import tensorflow as tf

In [5]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [6]:
help(tf.where)

Help on function where_v2 in module tensorflow.python.ops.array_ops:

where_v2(condition, x=None, y=None, name=None)
    Return the elements where `condition` is `True` (multiplexing `x` and `y`).
    
    This operator has two modes: in one mode both `x` and `y` are provided, in
    another mode neither are provided. `condition` is always expected to be a
    `tf.Tensor` of type `bool`.
    
    #### Retrieving indices of `True` elements
    
    If `x` and `y` are not provided (both are None):
    
    `tf.where` will return the indices of `condition` that are `True`, in
    the form of a 2-D tensor with shape (n, d).
    (Where n is the number of matching indices in `condition`,
    and d is the number of dimensions in `condition`).
    
    Indices are output in row-major order.
    
    >>> tf.where([True, False, False, True])
    <tf.Tensor: shape=(2, 1), dtype=int64, numpy=
    array([[0],
           [3]])>
    
    >>> tf.where([[True, False], [False, True]])
    <tf.Tensor: sh

In [7]:
# model.compile(loss=huber_fn, optimizer='adam')

In [8]:
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = threshold*tf.abs(error) - threshold**2/2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

In [10]:
# model.compile(loss=create_huber(2.0), optimizer='adam')

In [11]:
from tensorflow import keras

In [12]:
class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold*tf.abs(error) - self.threshold**2/2
        return tf.where(is_small_error, squared_loss, linear_loss)
    def get_config(self):
        base_config = super().get_config
        return {**base_config, "threshold":self.threshold}
    

In [13]:
# Custom Activation Functions, Initializers, Regularizers, and Constraint

In [14]:
def my_softplus(z):  # return value is just tf.nn.softplus(z)
    return tf.math.log(tf.exp(z) + 1)